<a href="https://colab.research.google.com/github/Kwannn666/GAN_AI/blob/main/Q36134255_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **正反方辯論機器人**

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


讀入標準套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

因為是使用 API 的方式呼叫, 所以要運行 Ollama Server, 而這邊是放在背景執行。
補充 : 如果在執行程式時有斷線的情況發生需要重新執行此程式碼

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


這邊我使用的是在 ollama 的兩個不同模型，分別為 gemma3:4b 跟 mistral

In [ ]:
# 拉取模型 A：gemma3:4b
!ollama pull gemma3:4b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling aeda25e63ebd... 100% ▕▏ 3.3 GB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling b6ae5839783f... 100% ▕▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
# 拉取模型 B：mistral
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea... 100% ▕▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 491dfa501e59... 100% ▕▏  801 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 42347cd80dc8... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [ ]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [ ]:
api_key = "ollama"

 OpenAI API 打開 `client` 的方式 , 預設服務 `port` 是 `11434`。

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3.正反方辯論機器人模型架構

記得角色 (role) 一共有三種, 分別是:

* `systemA`: 這是正方對話機器人的「人設」
* `systemB`: 這是反方對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [ ]:
model_A = "gemma3:4b"    # 正方
model_B = "mistral"     # 反方

正方對話機器人的「人設」:

"你是一位邏輯嚴謹、辯才無礙的正方辯論者，擅長站在議題的支持角度提出實證與理性觀點。請使用有說服力的語氣，避免太過冗長，一次只發表一個觀點,儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [ ]:
system_A = "你是一位邏輯嚴謹、辯才無礙的正方辯論者，擅長站在議題的支持角度提出實證與理性觀點。請使用有說服力的語氣，避免太過冗長，一次只發表一個觀點,儘量不要超過二十個字。請用台灣習慣的中文來回應。"

正方對話機器人的「人設」:

"你是一位尖銳反思、觀點犀利的反方辯論者，擅長質疑主流觀點並從反面切入議題。請使用有力的批判性語氣，每次一句論述即可，避免過長,儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [ ]:
system_B = "你是一位尖銳反思、觀點犀利的反方辯論者，擅長質疑主流觀點並從反面切入議題。請使用有力的批判性語氣，每次一句論述即可，避免過長,儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [ ]:
def init_debate_messages(user_prompt):
    messages_A = [
        {"role": "system", "content": system_A},
        {"role": "user", "content": user_prompt}
    ]
    messages_B = [
        {"role": "system", "content": system_B},
        {"role": "user", "content": user_prompt}
    ]
    return messages_A, messages_B


這邊我自訂了輸入，可以根據想讓兩個不同模型的對話機器人進行辯論的議題進行輸入

In [ ]:
prompt = input("請輸入辯論主題：")
messages_A, messages_B = init_debate_messages(prompt)


請輸入辯論主題：先有蛋還是先有雞?


這邊先讓正方發言，並且使用 `reply_A` 來擷取回應 ，並使用 `append` 分別讓正反方更新對話歷史

In [ ]:
response_A = client.chat.completions.create(
    model=model_A,
    messages=messages_A
)

reply_A = response_A.choices[0].message.content.strip()
print("正方：", reply_A)


messages_A.append({"role": "assistant", "content": reply_A})
messages_B.append({"role": "user", "content": reply_A})


正方： 科學證據顯示：先有蛋，並非基因突變的過程。


接著讓反方發言，並且使用 `reply_A` 來擷取回應 ，並同樣使用 `append` 分別讓正反方更新對話歷史

In [ ]:
response_B = client.chat.completions.create(
    model=model_B,
    messages=messages_B
)

reply_B = response_B.choices[0].message.content.strip()
print("反方：", reply_B)

messages_B.append({"role": "assistant", "content": reply_B})
messages_A.append({"role": "user", "content": reply_B})


反方： 先有蛋，進而生雞，更接近實際、可測量。基因突變？只是論述中的幻想故事。


使用 for 迴圈讓對話進行 9 次 並且讓正反方互相展開辯論

In [ ]:
# 進行後續 9 輪
for i in range(9):
    print(f"\n第 {i+2} 回合")

    # 反方回應
    response_B = client.chat.completions.create(
        model=model_B,
        messages=messages_B
    )
    reply_B = response_B.choices[0].message.content.strip()
    print("反方：", reply_B)
    messages_B.append({"role": "assistant", "content": reply_B})
    messages_A.append({"role": "user", "content": reply_B})

    # 正方回應
    response_A = client.chat.completions.create(
        model=model_A,
        messages=messages_A
    )
    reply_A = response_A.choices[0].message.content.strip()
    print("正方：", reply_A)
    messages_A.append({"role": "assistant", "content": reply_A})
    messages_B.append({"role": "user", "content": reply_A})



第 2 回合
反方： 你認為文化傳統之間哪些方面需要在全球化中重新思考？

文化各有千秋，在全球化過程中，需要回顧並重新思考以下三個方面：

1. 議題定位：將不同的文化問題定義成條件下通用化的標準，其實是忽略相關基礎情況、卻造成錯誤價值評価。在全球化中，我們應該更多使用對等、尊重差異的方式來回顧文化問題，並定位出相關問題空間。
2. 信息溝通：不同的文化背景造成了不同的信息溝通方式和挑戰，在全球化中要求我們更好地理解不同的文化差異，並學習更好地透過相關交流廣域來討論並處理問題。
3. 教育模式改革：在全球化中，我們需要將各種文化的知識和經驗融合來擁有更全面的認知和能力，而不是堅持單一文化的教育模式。此外，在全球化中，學習語言、文化相關的能力也是非常重要的，可以更好地理解和互動不同文化的人群。
正方： 尊重多元文化本體，而非刻意定義標準與價值。

第 3 回合
反方： 標準與價值不是一一對等的，文化區別可以讓我們更好地相互理解與沟通。
正方： 差異是溝通資本，相容性是新時代的基石。

第 4 回合
反方： 危險化差異，忽略兼容性，不足為今。
正方： 包容多元，合作共贏，務實至上。

第 5 回合
反方： 包容性是不會提升人類之力量。合作共利需求自本心出發。「務實」是無情的嗜金富玉。
正方： 理性與意志並行，平衡是永恆命題。

第 6 回合
反方： 「有時候，意志驅使人向前進步，理性則是制約之力，否則進步可能無法控制。」
正方： 平衡力量與理性，創造更穩健未來。

第 7 回合
反方： 在實際生活中，只能先做事再去求義，不然就無法建立基石，進而發展。反之亦然，如果據以義求實，難免過度讓理論和想象影響現實，造成混亂和違例。為了創造穩健的未來，實際生活是當前最重要的基石，而不是平衡力量與理性。
正方： 實踐至上，基業永續，務實是根本。

第 8 回合
反方： 在實際行動時，難免對現成價值產出影響。「基業永續」，無法完全隔離於實際，必須賦予其相互關係。如果只重視根本的「務實」，未來將無從補足。
正方： 穩健發展，層層遞進，基石即是提升。

第 9 回合
反方： 「強化主流觀點的維持，阻礙真正的進步。」
正方： 突破固有框架，創新思維，進步永無止境。

第 10 回合
反方： 在斷舊建新時，最多只能保留一小部分基石，不必對過往產出情感。
正方： 理性拆除過往，開創嶄新局面決

而是我實際使用下來發現只有對話的話很難去總結辯論結果以及梳理邏輯，所以又自己另外加上總結產生函數來產生總結

這邊我將雙方辯論紀錄彙整為文字，以便後續進行總結

In [ ]:
debate_text = []

for msg in messages_A:
    if msg["role"] == "assistant":
        debate_text.append(f"正方：{msg['content']}")
for msg in messages_B:
    if msg["role"] == "assistant":
        debate_text.append(f"反方：{msg['content']}")

debate_summary_input = "\n".join(debate_text)

總結的「人設」:

你是一位中立主持人，請針對以下雙方辯論做出總結，整理正反觀點，並提出你認為合理的結論，請用台灣習慣的中文來回應。"

In [ ]:
summary_prompt = [
    {"role": "system", "content": "你是一位中立主持人，請針對以下雙方辯論做出總結，整理正反觀點，並提出你認為合理的結論，請用台灣習慣的中文來回應。"},
    {"role": "user", "content": debate_summary_input}
]

由於我並未引入第三個模型進入作為總結機器人，而是使用了 mistral 模型做為總結機器人(也可視情況改為 ( gemma3:4b ) ，而這邊我不確定使用正方或者反方的機器人模型進行總結是否會有偏袒的行為產生，而這部分也是未來可以進行改進的(或者可以引入第三個模型進行總結)

In [ ]:
response_summary = client.chat.completions.create(
    model=model_B,  # or model_A (gemma3:4b)
    messages=summary_prompt
)

summary = response_summary.choices[0].message.content.strip()

print("\n 中立結論：\n")
print(summary)



 中立結論：

總結：

正方主要指出科學上存在先有蛋之法則、多元文化的尊重、創新思維、實際性和平衡力量與理性等優點，並強調應使用對等、尊重差異的方式來回顧文化問題，而反方主要強調在全球化中應讓不同文化問題各自定義標準、學習各種文化知識、提高信息溝通能力等重點。

正方認為實踐至上，突破現有框架，利用理性去解決問題，是創造更輕鬆、穩健的未來方法，而反方則認為對現存價值產出的影響和創新思想的限制，在實際生活中首先做實務再去求義才能建立基石，進而發展。

在斷舊建新時，反方強調只應保留一小部分基石，不必對過往產出情感。所以，根據這兩邊的論點，可以認為，在全球化中應該使用對等、尊重差異的方式來理解和回顧文化問題，並結合各種文化知識進行信息溝通，同時要注意先做實務再去求義，最後才能提升人類之力量。


### 4. 使用 gradio 套件打造 web app

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

對話機器人 app 設定

In [ ]:
title = "正反方辯論機器人"
description = "輸入一個議題，讓正方（gemma3:4b）與反方（mistral）進行辯論，各發言 10 回合，可選擇是否產出結論。"

model_A = "gemma3:4b"
model_B = "mistral"

system_A = "你是一位邏輯嚴謹、辯才無礙的正方辯論者，擅長站在議題的支持角度提出實證與理性觀點。請使用有說服力的語氣，避免太過冗長，一次只發表一個觀點,儘量不要超過二十個字。請用台灣習慣的中文來回應。"
system_B = "你是一位尖銳反思、觀點犀利的反方辯論者，擅長質疑主流觀點並從反面切入議題。請使用有力的批判性語氣，每次一句論述即可，避免過長,儘量不要超過二十個字。請用台灣習慣的中文來回應。"

from openai import OpenAI
client = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)

系統訊息不會改變，用於 State 初始化

In [ ]:
initial_messages_A = [{"role": "system", "content": system_A}]
initial_messages_B = [{"role": "system", "content": system_B}]

In [ ]:
state = gr.State(messages)

In [ ]:
import gradio as gr

state_A = gr.State(initial_messages_A.copy())
state_B = gr.State(initial_messages_B.copy())

在這邊請了 ChatGPT 4o 協助進行了將前面的對話邏輯程式碼整理為 Gradio 的形式 ，並且我使用了另一段 system ( modle_B Mistral ) 指示讓模型中立總結這段辯論，這部分原本是 GPT 沒有進行新增的，而是我實際使用下來發現只有對話的話很難去總結辯論結果以及梳理邏輯，所以又自己另外加上總結產生函數來產生總結

In [ ]:
def pipi(topic, rounds, with_summary):
    messages_A = [{"role": "system", "content": system_A}, {"role": "user", "content": topic}]
    messages_B = [{"role": "system", "content": system_B}]
    log = [f"📌 議題：{topic}\n"]

    # 正方先講
    res_A = client.chat.completions.create(model=model_A, messages=messages_A)
    rep_A = res_A.choices[0].message.content.strip()
    log.append(f"正方：{rep_A}\n")
    messages_A.append({"role": "assistant", "content": rep_A})
    messages_B.append({"role": "user", "content": rep_A})

    for _ in range(rounds):
        res_B = client.chat.completions.create(model=model_B, messages=messages_B)
        rep_B = res_B.choices[0].message.content.strip()
        log.append(f"反方：{rep_B}\n")
        messages_B.append({"role": "assistant", "content": rep_B})
        messages_A.append({"role": "user", "content": rep_B})

        res_A = client.chat.completions.create(model=model_A, messages=messages_A)
        rep_A = res_A.choices[0].message.content.strip()
        log.append(f"正方：{rep_A}\n")
        messages_A.append({"role": "assistant", "content": rep_A})
        messages_B.append({"role": "user", "content": rep_A})

    # 產出結論
    if with_summary:
        summary_prompt = "請根據上面雙方的辯論，幫我中立總結雙方觀點，並提出你認為合理的結論。"
        summary_input = [{"role": "system", "content": "你是一位中立主持人，請針對以下雙方辯論做出總結，整理正反觀點，並提出你認為合理的結論，請用台灣習慣的中文來回應"},
                         {"role": "user", "content": "\n".join(log)}]
        res_sum = client.chat.completions.create(model=model_B, messages=summary_input)
        summary = res_sum.choices[0].message.content.strip()
        log.append(f"\n結論：\n{summary}")

    return "\n".join(log)


In [ ]:
chatbot = gr.Textbox(label="辯論結果輸出區", lines=30)

這邊我新增了 `rounds = gr.Slider` 來讓使用者可以自由選擇想要看到幾此辯論結果 ( 最多10次 )

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## {title}\n{description}")

    topic = gr.Textbox(label="請輸入辯論議題", placeholder="例如：是否應該禁止 AI 創作藝術？")
    rounds = gr.Slider(1, 10, value=10, step=1, label="每方發言次數")
    summary = gr.Checkbox(label="是否產生總結", value=True)

    output = gr.Textbox(label="雙方辯論紀錄", lines=30)

    btn = gr.Button("開始辯論")
    btn.click(fn=pipi, inputs=[topic, rounds, summary], outputs=output)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0dc7921638d7007104.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
